# Actividad 4 Taller Pandas - Numpy 

## Instrucciones:

* Formula 1 pregunta de análisis sobre tu conjunto de datos
* Utilizando Pandas y Numpy:
* Limpieza de datos (Pandas: dropna, fillna, replace, etc.)
* Filtrado y selección de datos
* Transformación de columnas, creación de variables nuevas
* Agrupaciones y resúmenes (groupby)
* Conversión a array NumPy para operaciones matemáticas rápidas (media, desvío, normalización, operaciones vectorizadas)
* Responde la pregunta planteada con una breve interpretación.

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Análisis de Resultados de Clasificación de F1 base de datos: "qualifying_results_F1"

## Pregunta de Análisis
¿Cuál es la diferencia de rendimiento promedio entre los pilotos de las escuderías más históricas (Ferrari y McLaren) vs. las nuevas potencias (Red Bull y Mercedes) en términos de tiempo de clasificación (Q1) a lo largo de los años?

In [2]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('qualifying_results_F1.csv')
df.head(5)

,Season,Round,CircuitID,Position,DriverID,Code,PermanentNumber,GivenName,FamilyName,DateOfBirth,Nationality,ConstructorID,ConstructorName,ConstructorNationality,Q1,Q2,Q3
0,2000,1,albert_park,1,hakkinen,NaN,0,Mika,Häkkinen,1968-09-28,Finnish,mclaren,McLaren,British,1:30.556,0,0
1,2000,1,albert_park,2,coulthard,COU,0,David,Coulthard,1971-03-27,British,mclaren,McLaren,British,1:30.910,0,0
2,2000,1,albert_park,3,michael_schumacher,MSC,0,Michael,Schumacher,1969-01-03,German,ferrari,Ferrari,Italian,1:31.075,0,0
3,2000,1,albert_park,4,barrichello,BAR,0,Rubens,Barrichello,1972-05-23,Brazilian,ferrari,Ferrari,Italian,1:31.102,0,0
4,2000,1,albert_park,5,frentzen,NaN,0,Heinz-Harald,Frentzen,1967-05-18,German,jordan,Jordan,Irish,1:31.359,0,0


In [11]:
## PASO 1: Limpieza de Datos: Convertir los tiempos (Q1, Q2, Q3) a segundos totales

def time_to_seconds(time_str):
    """Convierte el formato 'MM:SS.mmm' a segundos totales (float)."""
    if pd.isna(time_str) or time_str == '0':
        return np.nan 
    try:
        minutes, rest = time_str.split(':')
        seconds = float(rest)
        return float(minutes) * 60 + seconds
    except:
        return np.nan

df['Q1_Seconds'] = df['Q1'].apply(time_to_seconds)

df['Q1_Seconds'] = df['Q1_Seconds'].fillna(
    df.groupby('Round')['Q1_Seconds'].transform('mean')
)

df.dropna(subset=['Q1_Seconds'], inplace=True)

In [12]:
## PASO 2: Filtrado y Creación de Variables: Asignación de 'Potencia' solo para las escuderías del análisis (Ferrari, McLaren, Red Bull y Mercedes)

escuderias_interes = ['Ferrari', 'McLaren', 'Red Bull', 'Mercedes']
df_filtered = df[df['ConstructorName'].isin(escuderias_interes)].copy()

def classify_team(constructor_name):
    if constructor_name in ['Ferrari', 'McLaren']:
        return 'Clásica'
    elif constructor_name in ['Red Bull', 'Mercedes']:
        return 'Potencia Reciente'
    return 'Otro' # Aunque ya filtramos, es una buena práctica

df_filtered['TeamType'] = df_filtered['ConstructorName'].apply(classify_team)

In [13]:
## PASO 3: Agrupaciones y Resúmenes: Agrupar por el Tipo de Equipo (TeamType) y calcular el promedio de Q1_Seconds

resumen_promedios = df_filtered.groupby('TeamType')['Q1_Seconds'].mean().reset_index()
resumen_promedios.rename(columns={'Q1_Seconds': 'TiempoPromedioQ1'}, inplace=True)

print("--- Resumen del Tiempo Promedio (Q1) por Tipo de Equipo (en segundos) ---")
print(resumen_promedios)
print("-" * 70)

--- Resumen del Tiempo Promedio (Q1) por Tipo de Equipo (en segundos) ---
            TeamType  TiempoPromedioQ1
0            Clásica         87.583772
1  Potencia Reciente         88.028783
----------------------------------------------------------------------


In [14]:
## PASO 4: Conversión a Array NumPy y Operaciones Vectorizadas

q1_array = df_filtered['Q1_Seconds'].values

media_global = np.mean(q1_array)
std_global = np.std(q1_array)

q1_normalized_array = (q1_array - media_global) / std_global

df_filtered['Q1_Normalized'] = q1_normalized_array

print(f"Media global de Q1 (NumPy): {media_global:.3f} segundos")
print(f"Desviación estándar global de Q1 (NumPy): {std_global:.3f} segundos")

Media global de Q1 (NumPy): 87.781 segundos
Desviación estándar global de Q1 (NumPy): 12.383 segundos


# Interpretación y Respuesta a la Pregunta de Análisis

Basándose en el análisis de los tiempos de clasificación (Q1) en los años analizados, se observa que el grupo de Potencias Recientes (Red Bull y Mercedes) tiene un tiempo de vuelta promedio de **88.028783** segundos, mientras que el grupo de Escuderías Clásicas (Ferrari y McLaren) registra un promedio de **87.583772** segundos. 

## Conclusión: 

Las Potencias Clásicas demuestran una ventaja de rendimiento promedio de aproximadamente **0.445011** segundos en la Q1 a lo largo del historial del dataset, sugiriendo una mayor consistencia y velocidad en la primera etapa de la clasificación.